## 코드 설계

1. question과 유사한 question 7개를 뽑는다.
2. 프롬프팅을 통해 question을 오지선다 문제로 만든다.
3. 프롬프팅을 통해 유사한 question의 answer을 변형한다. 이때 0~10 사이 점수를 부여한다.
4. 상위 4개의 오답을 선택한다.


# 0. 데이터 로드

In [1]:
from datasets import load_dataset
dataset = load_dataset("Cartinoe5930/raw_text_synthetic_dataset_50k", split = "train")

# 1. 역색인

In [2]:
from kiwipiepy import Kiwi
from collections import defaultdict
import math

In [3]:
class InvertedIndex:
    def __init__(self):
        self.reset()

    def reset(self):
        self.index = defaultdict(dict)
        self.kiwi = Kiwi()
        self.document_lengths = {}
        self.total_documents = 0
        self.average_document_length = 0
        self.documents = {}

    def tokenize(self, text):
        return [token.form for token in self.kiwi.tokenize(text)]

    def add_document(self, doc_id, question, answer, content='None'):
        tokens = self.tokenize(question)
        self.document_lengths[doc_id] = len(tokens)
        self.total_documents += 1
        self.documents[doc_id] = {'question': question, 'meta': {'answer': answer, 'content':content}}

        for token in set(tokens):
            if doc_id not in self.index[token]:
                self.index[token][doc_id] = 0
            self.index[token][doc_id] += tokens.count(token)

        self.average_document_length = sum(self.document_lengths.values()) / self.total_documents

    def calculate_bm25_score(self, query_tokens, doc_id):
        k1 = 1.5
        b = 0.75
        score = 0

        for token in query_tokens:
            if token not in self.index or doc_id not in self.index[token]:
                continue

            tf = self.index[token][doc_id]
            df = len(self.index[token])
            idf = math.log((self.total_documents - df + 0.5) / (df + 0.5) + 1)

            numerator = tf * (k1 + 1)
            denominator = tf + k1 * (1 - b + b * self.document_lengths[doc_id] / self.average_document_length)
            score += idf * numerator / denominator

        return score

    def search(self, query, k=5):
        query_tokens = self.tokenize(query)
        scores = defaultdict(float)

        for token in query_tokens:
            if token in self.index:
                for doc_id in self.index[token]:
                    scores[doc_id] += self.calculate_bm25_score(query_tokens, doc_id)

        top_k = sorted(scores.items(), key=lambda x: x[1], reverse = True)[:k]
        return [(doc_id, score, self.documents[doc_id]) for doc_id, score in top_k]

In [4]:
# add docs
from tqdm import tqdm

index = InvertedIndex()
for idx, data in enumerate(tqdm(dataset)):
    question = data['question']
    answer = data['response']
    index.add_document(idx, question, answer)

100%|██████████| 21900/21900 [00:46<00:00, 472.66it/s]


# 2. question 변형하기

In [112]:
question = dataset[1050]['question']
gold_answer = dataset[1050]['response']

print(question)
print(gold_answer)

재정 서비스 분야에서 규제를 준수하지 않는 불법적인 행위를 폭로한 내부 고발자들의 보호가 중요하다는 내용을 바탕으로, 내부 고발자들이 관련 정보를 제공함으로써 어떠한 방식으로 금융 시장의 신뢰성을 유지하고, 이들이 직면하는 개인적 및 직업적 결과로 인해 발생하는 윤리적 문제는 무엇인지 설명해 보시오.
내부 고발자들은 재정 서비스 분야에서 규제를 준수하지 않는 불법적인 행위를 폭로함으로써 금융 시장의 신뢰성을 유지하는 데 중요한 역할을 합니다. 이들은 기업의 불법적 또는 비윤리적 행동에 대한 정보를 제공함으로써, 소비자와 투자자들이 보다 신뢰할 수 있는 환경에서 의사결정을 할 수 있도록 돕습니다. 특히, 내부 고발자들의 폭로는 다음과 같은 방식으로 금융 시장의 신뢰성을 높이는 데 기여합니다:

1. **투명성 증가**: 불법 행위가 폭로됨으로써 기업의 운영이 투명해지며, 이에 따라 다른 기업들도 규제를 준수하려는 압박을 받게 됩니다.

2. **사후 조치 촉구**: 내부 고발자들이 제기한 문제들은 규제 기관이나 감사 기관에게 중요한 정보를 제공하여 조치를 취하도록 유도합니다. 이러한 사후 조치는 재정 안정을 유지하는 데 필수적입니다.

3. **윤리적 기준 강화**: 내부 고발자들의 고백은 점차적으로 기업 문화의 변화를 촉진하여, 장기적으로 모든 직원들이 윤리적 기준을 따르게 만드는 효과를 가져옵니다.

그러나 내부 고발자로서 정보를 제공하는 것은 개인적으로나 직업적으로 많은 위험을 동반하게 됩니다. 이들은 여러 가지 윤리적 문제와 함께 직면하게 되는데, 그 중 몇 가지는 다음과 같습니다:

1. **보복의 위험**: 내부 고발자들은 종종 고발 이후 개인적, 직업적 보복에 대한 두려움을 느낍니다. 회사에서 해고되거나 업무 환경이 악화될 수 있고, 이에 따라 경제적 손실이 발생할 수 있습니다.

2. **사회적 고립**: 고발 이후 동료들로부터 배척을 당하거나 신뢰를 잃을 수 있습니다. 이는 심리적인 스트레스와 우울감을 초래할 수 있습니다.

3. **윤리적 딜레

In [71]:
import openai
from dotenv import load_dotenv
import os

dotenv_path = os.path.join(os.getcwd(), 'KRX', '.env')
load_dotenv(dotenv_path)

openai_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI(api_key=openai_key)

In [98]:
question_answer_translate_system_prompt ="""Question and gold answer are given.
The given question and answer should be converted into a multiple choice question and one correct answer choice.
The question must be converted into a form of 'what is right.'.
The given answer should be converted into a concise and unquestionable answer.
Give a transformed question and a transformed gold answer.
please write in korean.
""" 
question_answer_translate_user_prompt="""
### Question: {}
### Gold Answer: {}

### Transformed Question: 
### Transformed Gold Answer:
"""


In [106]:
# 8.27 -> ... 와우 8.27
qat_system_prompt = {'role':'system', 'content': question_answer_translate_system_prompt}
qat_user_prompt = {'role':'user', 'content': question_answer_translate_user_prompt.format(question, gold_answer)}

response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages = [qat_system_prompt, qat_user_prompt]
)

result = response.choices[0].message.content
print(result)


### Transformed Question:  
AI 시스템이 고품질 훈련 데이터를 확보하기 위해 필요한 조치는 무엇입니까?

### Transformed Gold Answer:  
AI 시스템이 고품질 훈련 데이터를 확보하기 위해 필요한 조치는 데이터 수집의 다양성 확보, 데이터 품질 검증, 윤리적 데이터 사용 원칙 수립, 편향 탐지 및 수정 메커니즘 도입, 지속적인 학습 및 검토입니다.


In [108]:
import re

pattern = r"### Transformed Question:(.*?)(?=### Transformed Gold Answer:)"
match = re.search(pattern, result, re.DOTALL)
if match:
    transformed_question = match.group(1).strip()

answer_pattern = r'### Transformed Gold Answer:(.*)'
answer_match = re.search(answer_pattern, result, re.DOTALL)

if answer_match:
    transformed_gold_answer = answer_match.group(1).strip()
    
print(transformed_question)
print(transformed_gold_answer)

AI 시스템이 고품질 훈련 데이터를 확보하기 위해 필요한 조치는 무엇입니까?
AI 시스템이 고품질 훈련 데이터를 확보하기 위해 필요한 조치는 데이터 수집의 다양성 확보, 데이터 품질 검증, 윤리적 데이터 사용 원칙 수립, 편향 탐지 및 수정 메커니즘 도입, 지속적인 학습 및 검토입니다.


# 유사한 자료 모으기

- question 을 비교하는게 맞을까??


In [104]:
similar_questions = index.search(transformed_question, k=10)
print(similar_questions)

[]


# 오답 생성하기

In [102]:
wrong_answer_system_prompt = """You will given a question, gold answer and one irrelevant answer.
Your job is to return a well-designed wrong answer to the Question by referring to Irrelevant Answer.
Also you will give the generated Good Wrong Answer a score between 0 and 100.
well-designed wrong answer should satisfy:
1. You must not generate incorrect answers that are outside the subject of the question.
2. Replacing references to multiple companies as companies interested in your questions.
3. Good Wrong Answer confuses people of choosing compared to Gold Answer, but it should be no doubt that it's a wrong answer.
4. The length of Generated Good Wrong Answer must be similar compared to the legnth of gold answer.
"""

wrong_answer_user_prompt = """
### Question: {}
### Gold Answer: {}
### Irrelevant Answer: {}
### Good Wrong Answer:
### Score:
"""

In [103]:
# 8.27 -> 8.27 싸다
selections = []

for item in similar_questions:
    irrelevant_answer = item[2]['meta']['answer']
    prompt = wrong_answer_user_prompt.format(transformed_question, transformed_gold_answer, irrelevant_answer)

    instruction_msg = {"role":"system", "content": wrong_answer_system_prompt}
    user_msg = {"role":'user', "content":prompt}

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages = [instruction_msg, user_msg]
    )

    selections.append(response.choices[0].message.content)

In [95]:
print('문제: ', transformed_question)
print('정답: ', transformed_gold_answer)
print()
for idx, item in enumerate(selections):
    print(idx+1, end='')
    print("-" * 100)
    print(item)

문제:  금융 서비스 조직이 AI 시스템을 설계할 때 고품질 훈련 데이터를 확보하기 위해 필요한 조치는 무엇인가?
정답:  고품질 훈련 데이터를 확보하기 위해서는 데이터 수집의 다양성 확보, 데이터 품질 검증, 윤리적 데이터 사용 원칙 수립, 편향 탐지 및 수정 메커니즘 도입, 지속적인 학습 및 검토가 필요하다.

1----------------------------------------------------------------------------------------------------
고품질 훈련 데이터를 확보하기 위해 금융 서비스 조직은 다음과 같은 조치를 취해야 합니다:

1. **소셜 미디어 데이터 활용**:
   - 다양한 인종, 성별, 경제적 배경의 사람들을 대상으로 소셜 미디어 플랫폼에서 수집된 데이터를 활용하여 데이터셋의 대표성을 높입니다. 이는 AI 모델이 특정 그룹에 더욱 집중되도록 돕습니다.

2. **데이터 검토 절차 강화**:
   - 수집된 데이터를 활용하여 정기적으로 사용자 반응을 분석하고, 이를 통해 적합성을 점검하는 프로세스를 설정합니다. 이는 소셜 미디어 활동을 기반으로 하며, 변동성이 큰 데이터를 민감하게 다뤄야 합니다.

3. **개인 정보 활용 기준 수정**:
   - 데이터 수집을 위해 개인의 동의 없이 소셜 미디어 정보 등을 활용하는 기준을 마련합니다. 이는 소비자의 의견을 반영하려는 노력을 통해 이해관계자 간의 조율을 비롯한 문제를 최소화합니다.

4. **결과의 편향 확인**:
   - AI가 생성한 결과가 편향되지 않도록 소셜 미디어에서 발생하는 다양한 의견을 반영하여, 공정한 결과를 도출하는 검증 알고리즘을 활용합니다.

5. **AI의 지속적 업데이트**:
   - AI 시스템은 수집된 소셜 미디어 데이터를 바탕으로 정기적으로 업데이트되며, 실시간 반응을 통해 모델 성능을 향상시키는 방향으로 조정됩니다.

이와 같은 조치들은 특히 인종 간 신용 점수 구축 시 사용될 수 있으며, 그 결과 다양한 의견을 반

# 문제 생성

In [79]:
import heapq

wrong_selections = []
wrong_selections_heap = []
answer_pattern = r'### Good Wrong Answer:(.*?)(?=\n\n### Score:)'
score_pattern = r'### Score:\s*(\d+)'

for select in selections:
    answer_match = re.search(answer_pattern, select, re.DOTALL)
    if answer_match:
        wrong_select_text = answer_match.group(1).strip()
        
    score_match = re.search(score_pattern, select)
    if score_match:
        wrong_select_score = score_match.group(1)
    
    heapq.heappush(wrong_selections_heap, (-int(wrong_select_score), wrong_select_text))

for _ in range(4):
    _, text = heapq.heappop(wrong_selections_heap)
    wrong_selections.append(text)

In [80]:
for idx, text in enumerate(wrong_selections):
    print(f"{idx}. {text}")

0. 금융 서비스 조직이 인공지능(AI) 시스템 설계 시, 고품질 훈련 데이터 확보를 위해 필요한 조치는 데이터 수집의 단순화를 통해 특정 데이터 세트만 취합하고, AI 모델의 접근성을 높이는 데 집중해야 합니다. 특히, 데이터의 대표성은 무시하고 편향 탐지 메커니즘을 사용하지 않는 것이 좋습니다. 이를 통해 AI는 특정 그룹의 사용자에게만 최적화된 결과를 제공함으로써, 금융 서비스의 신뢰성을 높이게 됩니다.
1. 금융 서비스 조직이 인공지능(AI) 시스템 설계 시, 고품질 훈련 데이터 확보를 위해서는 대량의 데이터를 신속하게 수집하고 처리하는 것이 중요하다. 이러한 시스템은 의사 결정의 효율성을 증가시키고, 다양한 금융 거래의 통찰력을 제공함으로써 고객에게 더 나은 서비스를 제공할 수 있다. 그러나 단순히 데이터를 많이 수집하는 것만으로는 충분하지 않으며, 윤리적 기준을 포함한 데이터 사용 원칙을 확립하는 것이 필수적이다. 따라서 데이터의 품질을 강화하기 위해서는 다양한 출처에서 데이터가 수집되어야 하며, 이 과정에서 고객의 신뢰도를 높일 수 있는 방법을 모색해야 한다.
2. 금융 서비스 조직이 인공지능(AI) 시스템 설계 시, 훈련 데이터와 테스트 데이터의 역할을 명확히 이해하고, 과적합 문제를 해결하기 위한 여러 기법을 도입하는 것이 중요하다. 예를 들어, 훈련 데이터의 변형을 통해 데이터 양을 늘림으로써 모델의 일반화 능력을 높일 수 있으며, 정규화 기법을 활용하여 모델의 복잡성을 줄여야 한다. 또한, 데이터의 분할을 통해 교차 검증을 실시하여 모델의 성능을 평가하는 것이 필요하다. 이는 데이터 수집의 다양성 및 품질 확보와는 별개로, AI 시스템의 효과성을 높이는 데 기여한다.
3. 금융 서비스 조직이 AI 시스템의 훈련 데이터의 편향 문제를 해결하기 위해서는 데이터베이스의 외부 검증을 강화하고, 기존의 알고리즘을 수정하는 작업에 집중해야 합니다. 이를 통해 데이터의 품질을 높이는 것이 아니라 오히려 외부에서 얻은 데이터의 신뢰성을 보장하고 기존의 사

In [81]:
import random
import pandas as pd

answer_num = random.randint(0, 4)

final_choices = []
for i in range(5):
    if i == answer_num:
        final_choices.append(transformed_gold_answer)
    else:
        final_choices.append(wrong_selections.pop())


item = {'question': transformed_question, 'choices': final_choices, 'answer': answer_num}
print(item)




{'question': '금융 서비스 조직이 인공지능(AI) 시스템 설계 시, 고품질 훈련 데이터 확보를 위해 필요한 조치로 옳은 것은 무엇인가?', 'choices': ['데이터 수집의 다양성 확보, 데이터 품질 검증, 윤리적 데이터 사용 원칙 수립, 편향 탐지 및 수정 메커니즘 도입, 지속적인 학습 및 검토가 필요하다.', '금융 서비스 조직이 AI 시스템의 훈련 데이터의 편향 문제를 해결하기 위해서는 데이터베이스의 외부 검증을 강화하고, 기존의 알고리즘을 수정하는 작업에 집중해야 합니다. 이를 통해 데이터의 품질을 높이는 것이 아니라 오히려 외부에서 얻은 데이터의 신뢰성을 보장하고 기존의 사용자 데이터를 바탕으로 알고리즘을 보완해야 합니다. 마지막으로, 특정 집단의 데이터만을 중심으로 훈련하도록 하여 의사결정 과정이 간소화될 수 있도록 해야 합니다.', '금융 서비스 조직이 인공지능(AI) 시스템 설계 시, 훈련 데이터와 테스트 데이터의 역할을 명확히 이해하고, 과적합 문제를 해결하기 위한 여러 기법을 도입하는 것이 중요하다. 예를 들어, 훈련 데이터의 변형을 통해 데이터 양을 늘림으로써 모델의 일반화 능력을 높일 수 있으며, 정규화 기법을 활용하여 모델의 복잡성을 줄여야 한다. 또한, 데이터의 분할을 통해 교차 검증을 실시하여 모델의 성능을 평가하는 것이 필요하다. 이는 데이터 수집의 다양성 및 품질 확보와는 별개로, AI 시스템의 효과성을 높이는 데 기여한다.', '금융 서비스 조직이 인공지능(AI) 시스템 설계 시, 고품질 훈련 데이터 확보를 위해서는 대량의 데이터를 신속하게 수집하고 처리하는 것이 중요하다. 이러한 시스템은 의사 결정의 효율성을 증가시키고, 다양한 금융 거래의 통찰력을 제공함으로써 고객에게 더 나은 서비스를 제공할 수 있다. 그러나 단순히 데이터를 많이 수집하는 것만으로는 충분하지 않으며, 윤리적 기준을 포함한 데이터 사용 원칙을 확립하는 것이 필수적이다. 따라서 데이터의 품질을 강화하기 위해서는 다양한 출처에서 데이터가 수집되어야

In [82]:
print(item['question'])

for idx, c in enumerate(item['choices']):
    print(f"{idx}. {c}")

print("정답 : ", item['answer'])

금융 서비스 조직이 인공지능(AI) 시스템 설계 시, 고품질 훈련 데이터 확보를 위해 필요한 조치로 옳은 것은 무엇인가?
0. 데이터 수집의 다양성 확보, 데이터 품질 검증, 윤리적 데이터 사용 원칙 수립, 편향 탐지 및 수정 메커니즘 도입, 지속적인 학습 및 검토가 필요하다.
1. 금융 서비스 조직이 AI 시스템의 훈련 데이터의 편향 문제를 해결하기 위해서는 데이터베이스의 외부 검증을 강화하고, 기존의 알고리즘을 수정하는 작업에 집중해야 합니다. 이를 통해 데이터의 품질을 높이는 것이 아니라 오히려 외부에서 얻은 데이터의 신뢰성을 보장하고 기존의 사용자 데이터를 바탕으로 알고리즘을 보완해야 합니다. 마지막으로, 특정 집단의 데이터만을 중심으로 훈련하도록 하여 의사결정 과정이 간소화될 수 있도록 해야 합니다.
2. 금융 서비스 조직이 인공지능(AI) 시스템 설계 시, 훈련 데이터와 테스트 데이터의 역할을 명확히 이해하고, 과적합 문제를 해결하기 위한 여러 기법을 도입하는 것이 중요하다. 예를 들어, 훈련 데이터의 변형을 통해 데이터 양을 늘림으로써 모델의 일반화 능력을 높일 수 있으며, 정규화 기법을 활용하여 모델의 복잡성을 줄여야 한다. 또한, 데이터의 분할을 통해 교차 검증을 실시하여 모델의 성능을 평가하는 것이 필요하다. 이는 데이터 수집의 다양성 및 품질 확보와는 별개로, AI 시스템의 효과성을 높이는 데 기여한다.
3. 금융 서비스 조직이 인공지능(AI) 시스템 설계 시, 고품질 훈련 데이터 확보를 위해서는 대량의 데이터를 신속하게 수집하고 처리하는 것이 중요하다. 이러한 시스템은 의사 결정의 효율성을 증가시키고, 다양한 금융 거래의 통찰력을 제공함으로써 고객에게 더 나은 서비스를 제공할 수 있다. 그러나 단순히 데이터를 많이 수집하는 것만으로는 충분하지 않으며, 윤리적 기준을 포함한 데이터 사용 원칙을 확립하는 것이 필수적이다. 따라서 데이터의 품질을 강화하기 위해서는 다양한 출처에서 데이터가 수집되어야 하며, 이 과정에서 고객의 신뢰도를 높일 수 